# 📖 1. Импорт библиотек


In [1]:
import fitz
import pandas as pd
import io
import glob
import os
from datetime import datetime
import re

# 📁 2. Загрузка PDF файлов

In [2]:
# загрузка в list пути всех нужных pdf файлов
all_pdf = glob.glob(r"PDF2\*.pdf")

In [3]:
len(all_pdf)

30

In [4]:
all_pdf

['PDF2\\Cocoapods, Carthage, SPM как выбрать менеджер зависимостей в iOS.pdf',
 'PDF2\\Deutsche Telekom и Perplexity объявили о новом «AI Phone» стоимостью менее 1 000 долларов _ Хабр.pdf',
 'PDF2\\imgonline-com-ua-site2pdfB28eKGLy6EjA.pdf',
 'PDF2\\OpenAI отходит от теории внезапного прорыва в области AGI _ Хабр.pdf',
 'PDF2\\OSINT & Hacking — как работает фишинг для нельзяграма _ Хабр3.pdf',
 'PDF2\\Yandex N.V. запретили в\xa0течение пяти лет создавать аналогичные «Яндексу» сервисы _ Хабр.pdf',
 'PDF2\\Аннотации типов в Python_ коротко о главном _ Хабр.pdf',
 'PDF2\\Биткоин включили в пул стратегического резерва США _ Хабр.pdf',
 'PDF2\\Быстрое начало работы с Gitlab CICD.pdf',
 'PDF2\\Генеральный директор Mozilla покинула свой пост _ Хабр.pdf',
 'PDF2\\Гилморовская модификация стратокастера _ Хабр.pdf',
 'PDF2\\За что безопасники будут гореть в аду_ _ Хабр.pdf',
 'PDF2\\Заезжаем в Kotlin Multiplatform. Но какой ценой_ _ Хабр.pdf',
 'PDF2\\ИИ-агенты в Альфа-Банке_ нейросети создают а

# 3. Парсинг 📤


In [5]:
def pars_pdf(file_path):
    file_name = file_path.split('\\')[-1][:-4]
    pdf = fitz.open(file_path)
    text = []
    
    for page_num in range(len(pdf)):
        page_text = pdf.load_page(page_num).get_text("text")
        text += page_text.split('\n')
    
    name_company = ""
    raiting = ""
    activity = ""
    date_publish = ""
    text_article = ""
    
    date_string = pdf.metadata.get("creationDate", "")[2:10]
    if date_string and not str.isspace(date_string):
        date_publish = datetime.strptime(date_string, "%Y%m%d").date()
    else:
        timestamp = os.path.getmtime(file_path)
        date_publish = datetime.fromtimestamp(timestamp).date()
    
    try:
        raiting_index = text.index("Рейтинг")
        raiting = text[raiting_index - 1] if raiting_index > 0 else "Нет оценок"
        name_company = text[raiting_index + 1] if raiting_index + 1 < len(text) else "Нет название"
        activity = text[raiting_index + 2] if raiting_index + 2 < len(text) else "Нет описания"
        
        if activity == "Подписаться":
            activity = "Нет описания"
        
        full_text = ' '.join(text)
        activity_pos = full_text.find(activity)
        
        if activity_pos != -1:
            text_article = full_text[activity_pos + len(activity):].strip()
        else:
            text_article = full_text
        
    except ValueError:
        raise Exception(f"\"{file_name}\" - рейтинг не обнаружен")
    
    return file_name, name_company, raiting, activity, date_publish, text_article

In [6]:
# Перебор текста из pdf файла, парсинг
Brak = []
NameCompany = []
Rating = []
DataPublish = []
Activity = []
TextArticle = []
Title = []
for pdf in all_pdf:
    try:  
        tt, N, R, A, D, T = pars_pdf(pdf)
        if D == 'Подписаться':
            D = 'Нет даты'
        Title.append(tt)
        NameCompany.append(N)
        Rating.append(R)
        DataPublish.append(D)
        Activity.append(A)
        TextArticle.append(T)
    except Exception as e:
        print(e)
        Brak.append(pdf)

"Yandex N.V. запретили в течение пяти лет создавать аналогичные «Яндексу» сервисы _ Хабр" - рейтинг не обнаружен
"За что безопасники будут гореть в аду_ _ Хабр" - рейтинг не обнаружен
"Заезжаем в Kotlin Multiplatform. Но какой ценой_ _ Хабр" - рейтинг не обнаружен
"Компании _ Хабр" - рейтинг не обнаружен
"Компании" - рейтинг не обнаружен
"Нейросети в разработке_ мощный инструмент или слепая вера_ _ Хабр" - рейтинг не обнаружен
"Правительство не продлило мораторий на выездные налоговые проверки IT‑компаний _ Хабр" - рейтинг не обнаружен


In [7]:
Brak

['PDF2\\Yandex N.V. запретили в\xa0течение пяти лет создавать аналогичные «Яндексу» сервисы _ Хабр.pdf',
 'PDF2\\За что безопасники будут гореть в аду_ _ Хабр.pdf',
 'PDF2\\Заезжаем в Kotlin Multiplatform. Но какой ценой_ _ Хабр.pdf',
 'PDF2\\Компании _ Хабр.pdf',
 'PDF2\\Компании.pdf',
 'PDF2\\Нейросети в разработке_ мощный инструмент или слепая вера_ _ Хабр.pdf',
 'PDF2\\Правительство не\xa0продлило мораторий на\xa0выездные налоговые проверки IT‑компаний _ Хабр.pdf']

In [8]:
# Создание DataFrame
df = pd.DataFrame({
'title': [],
'name_company': [],
'rating': [],
'date_publish': [],
'activity': [],
'text_article': []
})

In [9]:
print(len(Title), len(NameCompany), len(Rating), len(DataPublish),len(Activity), len(TextArticle) )

23 23 23 23 23 23


In [10]:
df['title'] = Title
df['name_company'] = NameCompany
df['rating'] = Rating
df['date_publish'] = DataPublish
df['activity'] = Activity
df['text_article'] = TextArticle

In [11]:
df

,title,name_company,rating,date_publish,activity,text_article
0,"Cocoapods, Carthage, SPM как выбрать менеджер ...",red_mad_robot,117.94,2023-03-20,№1 в разработке цифровых решений для бизнеса,6 часов назад Средний18 мин redmadrobot Cocoap...
1,Deutsche Telekom и Perplexity объявили о новом...,Технократия @technokratiya,4.4,2025-03-03,ИТ-Компания,Сайт Сайт Тем не менее это свидетельствует о т...
2,imgonline-com-ua-site2pdfB28eKGLy6EjA,Cloud4Y,71.07,2023-03-20,#1 Корпоративный облачный провайдер,Автор оригинала: Yashwant Singh Взлом Instagra...
3,OpenAI отходит от теории внезапного прорыва в ...,BotHub,459.2,2025-03-07,"Агрегатор нейросетей: ChatGPT, Claude, Midjourney","Подписаться OpenAI - компания, стоящая за Chat..."
4,OSINT & Hacking — как работает фишинг для нель...,Cloud4Y,71.07,2023-03-20,#1 Корпоративный облачный провайдер,Автор оригинала: Yashwant Singh Взлом Instagra...
5,Аннотации типов в Python_ коротко о главном _ ...,OTUS,717.61,2025-03-13,Цифровые навыки от ведущих экспертов,OTUS Цифровые навыки от ведущих экспертов Сайт...
6,Биткоин включили в пул стратегического резерва...,6 марта 2025 года Дональд Трамп подписал указ ...,605.3,2025-03-07,в национальный криптовалютный резерв США. Попо...,"планируют криптомонетами, которые правоохранит..."
7,Быстрое начало работы с Gitlab CICD,Southbridge,280.79,2023-03-20,Обеспечиваем стабильную работу highload-проектов,Автор оригинала: Seifeldin Mahjoub Перевели ст...
8,Генеральный директор Mozilla покинула свой пос...,@AnnieBronson,145.1,2024-02-08,Информационная служба Хабра,Реклама Комментарии Здесь пока нет ни одного к...
9,Гилморовская модификация стратокастера _ Хабр,RUVDS.com,2656.49,2025-03-13,VDS/VPS-хостинг. Скидка 15% по коду HABR15,RUVDS.com VDS/VPS-хостинг. Скидка 15% по коду ...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         23 non-null     object
 1   name_company  23 non-null     object
 2   rating        23 non-null     object
 3   date_publish  23 non-null     object
 4   activity      23 non-null     object
 5   text_article  23 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB


In [13]:
import pandas
df.to_csv("pdfdf.csv", index=False)

In [15]:
df2 = pd.read_csv("pdfdf.csv")
df2.sample(10)

,title,name_company,rating,date_publish,activity,text_article
18,Новая модель Alibaba с открытым исходным кодом...,BotHub,459.20,2025-03-07,"Агрегатор нейросетей: ChatGPT, Claude, Midjourney","Подписаться Qwen Team , подразделение китайско..."
7,Быстрое начало работы с Gitlab CICD,Southbridge,280.79,2023-03-20,Обеспечиваем стабильную работу highload-проектов,Автор оригинала: Seifeldin Mahjoub Перевели ст...
20,Новый пузырь доткомов Кремниевой долины _ Хабр,RUVDS.com,2656.49,2025-03-13,VDS/VPS-хостинг. Скидка 15% по коду HABR15,RUVDS.com VDS/VPS-хостинг. Скидка 15% по коду ...
13,Как OSA превращает пустые полки в полные корзи...,Magnit Tech,63.19,2025-03-13,Соединяем IT и ритейл,Magnit Tech Соединяем IT и ритейл Сайт Хабр Ка...
14,Как системному аналитику написать хорошее резю...,@Systems_Education,0.00,2023-03-22,Пользователь,Сайт Сайт Facebook Telegram Telegram Реклама К...
6,Биткоин включили в пул стратегического резерва...,6 марта 2025 года Дональд Трамп подписал указ ...,605.30,2025-03-07,в национальный криптовалютный резерв США. Попо...,"планируют криптомонетами, которые правоохранит..."
21,Почему Data Science не для вас,Tourmaline Core @TourmalineCore,2.00,2023-03-20,Пользователь,Комментарии 5 Публикации • Data Science – про ...
22,Школа программирования 8 класс_ ТОП-10 школ _ ...,IT для детей,292.43,2025-03-13,IT для детей,IT для детей Компания Сайт 1 Карма 50 Рейтинг ...
9,Гилморовская модификация стратокастера _ Хабр,RUVDS.com,2656.49,2025-03-13,VDS/VPS-хостинг. Скидка 15% по коду HABR15,RUVDS.com VDS/VPS-хостинг. Скидка 15% по коду ...
2,imgonline-com-ua-site2pdfB28eKGLy6EjA,Cloud4Y,71.07,2023-03-20,#1 Корпоративный облачный провайдер,Автор оригинала: Yashwant Singh Взлом Instagra...
